### Medicines_Identifiers

In [3]:
import spacy

In [4]:
from tqdm import tqdm
import spacy
import random
import json
import re

In [5]:
training_data = []
lines=[]
with open('New_Medical_reports.json', 'r',encoding="utf8") as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    entities = []
    if(data['annotation'] is not None):
        for annotation in data['annotation']:
              if(annotation is not None):
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))

    if(entities != []):
        training_data.append((text, {"entities" : entities}))

In [6]:
len(training_data)

50

In [7]:
training_data[4:15]

[('Hello,Siddhanth Singh of age 25 years.Your have symptoms like a food intolerance,adverse reaction to a medication,viral infection and bacterial  infection.The diagnosis report says that you have diarrhea disease.I am prescribing you Attapulgite,Bismuth Subsalicylate,Charcoal Activated,Difenoxin and Atropine medicine.You should wash the cooking and food preparation areas more frequently,take food immediately after preparing it.',
  {'entities': [(286, 308, 'Medicines'),
    (267, 285, 'Medicines'),
    (245, 266, 'Medicines'),
    (233, 244, 'Medicines')]}),
 ('Hi, Pratik Kumar of age 25 years.Your symptoms are  pain,redness, feeling like the eye is scratched or something is in it, sensitivity to light, andblurry vision.The diagnosis report confirms that you have the disease Corneal Ulcers.I am prescibing you the medicine Antibacterials (Ophthalmic).I recommend you to wear eye protection when using power tools .You should also try to do exercise and yoga.',
  {'entities': [(266, 293,

In [8]:
## Training a blank spacy english model.

nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 50/50 [00:00<00:00, 48975.99it/s]


Statring iteration 0
{'ner': 929.9250523526625}
Statring iteration 1
{'ner': 781.015163274015}
Statring iteration 2
{'ner': 938.7415757498438}
Statring iteration 3
{'ner': 243.73678143274017}
Statring iteration 4
{'ner': 94.09807148690405}
Statring iteration 5
{'ner': 81.89931690583558}
Statring iteration 6
{'ner': 47.408823278910155}
Statring iteration 7
{'ner': 28.744351885924555}
Statring iteration 8
{'ner': 14.415915395399619}
Statring iteration 9
{'ner': 10.918812400253655}
Statring iteration 10
{'ner': 13.64042177952682}
Statring iteration 11
{'ner': 9.246584695489119}
Statring iteration 12
{'ner': 38.184494395582675}
Statring iteration 13
{'ner': 24.9850840686269}
Statring iteration 14
{'ner': 21.001477069965905}
Statring iteration 15
{'ner': 14.82909787500828}
Statring iteration 16
{'ner': 3.1398534285092046}
Statring iteration 17
{'ner': 20.73502808041312}
Statring iteration 18
{'ner': 20.841078488360182}
Statring iteration 19
{'ner': 38.58165506906669}


In [9]:
nlp.to_disk("new_model_4")

In [11]:
import spacy
nlp4=spacy.load("new_model_4")
doc=nlp4(open("./MEDICAL_REPORTS/TEST_REPORT_11.txt").read())

In [12]:
print(doc)

hello,Piyush Jagga, You are 25 years old.According to your problems i have read you are having the problems of Persistent lumps,swollen glands.You are suffering from problem of disease HIV.Medicines diazepam Gel or rectal solution: 5 mg/ mL in 0.5 mL 2‐ mL 4‐mL tubes,lorazepam Parenteral formulation: 2 mg/ mL in 1‐ mL ampoule; 4 mg/ mL in 1‐ mL ampoule.avoid lifting heavy weights.You should maintain your Body Mass Index.I would suggest doing more of effort.


In [13]:

for entity in doc.ents:
    print(entity.text)
#         i+=1
#     else:
#         break

Piyush Jagga
diazepam Gel or
: 5 mg/ mL in 0.5 mL 2‐ mL
Parenteral
: 2 mg/ mL in 1‐ mL
mg/ mL in 1‐ mL ampoule.avoid
